### read data and calc returns

In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import os
from datetime import datetime

def combine_csv_files(directory_path):
    # List to store individual dataframes
    dfs = []

    # Loop through all files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv') and filename.startswith('nootc_'):
            # Extract date from filename
            date_str = filename.replace('nootc_', '').replace('.csv', '')

            # Read the CSV file
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)

            # Add date column
            df['DATE'] = pd.to_datetime(date_str)

            # Append to list of dataframes
            dfs.append(df)

    # Concatenate all dataframes
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        return combined_df
    else:
        return None

In [3]:
df = combine_csv_files(directory_path = '../p123/data/api')

In [4]:
df = df.dropna(subset = ['tickers'])

In [5]:
dfnan_counts = df.groupby(['DATE']).apply(lambda x: x.isna().sum())

/var/folders/sw/kl2bvz9d5275zz0p278kz9nc0000gn/T/ipykernel_97285/3733439746.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dfnan_counts = df.groupby(['DATE']).apply(lambda x: x.isna().sum())


In [6]:
list(df.columns)

['p123Uids',
 'tickers',
 'names',
 'MktCap',
 'Close',
 'Open',
 'High',
 'Low',
 'Volume',
 'ret_1w',
 'frel_3m',
 'frel_1y',
 'Beta1Y',
 'price_vol_1y',
 'share_turnover_3m',
 'residual_return_1m',
 '3MoRS',
 '3MoRS3MoAgo',
 '3MoRS6MoAgo',
 '3MoRS9MoAgo',
 'PR2SalesTTM',
 'EarnYield',
 'Pr2BookQ',
 'Pr2TanBkQ',
 'FCFYield',
 '$unlevered_fcfToEV',
 'OcfToEV',
 'EV2SalesTTM',
 'GrossProfitToEV',
 'EV2EBITDATTM',
 'EPSReportedPctGrPYQ',
 'EPSReportedAccelPYQTTM',
 'EPSReportedPctGrTTM',
 'SalesActualGr%PYQ',
 'SalesActualGr%TTM',
 'SalesActualAccelPYQTTM',
 'DbtTot2CapQ',
 'IntCovTTM',
 'AstTurnTTM',
 'OpMgn%TTM',
 'ROI%Q',
 'ROE%Q',
 'Div%ChgTTM',
 'Yield',
 'net_payout_ratio',
 'CurFYEPS_pctchg4WK',
 'CurQEPS_pctchg4WK',
 'CurQEPS_estvar',
 'CurFYEPS_estvar',
 'SalesSurp%Q1',
 'Surprise%Q1',
 'fwd_ebitdantm_ev',
 'fwd_ebitdaq_ev',
 'epsg_pyq_est_act',
 'DATE']

In [7]:



drop_cols1 = [
    'p123Uids',
    'names',
    'Close',
    'Open',
    'High',
    'Low',
    'Volume',
    'frel_3m',
    'frel_1y',
    
]

drop_cols1 = [ c for c in drop_cols1 if c in df.columns.tolist() ]

In [8]:

df = df.drop(columns =  drop_cols1)
df = df.rename(columns = {'tickers':'ID',
                          'ret_1w':'stock_return'})

In [9]:
# stats and returns are as of given date
# so either shift returns up, or stats down to do regression on returns vs stats as of beginning of returns period 
# lets shift stats down to align with returns end date

In [10]:
# shift down everything except ID, DATE, stock_return 
df.columns.tolist()

['ID',
 'MktCap',
 'stock_return',
 'Beta1Y',
 'price_vol_1y',
 'share_turnover_3m',
 'residual_return_1m',
 '3MoRS',
 '3MoRS3MoAgo',
 '3MoRS6MoAgo',
 '3MoRS9MoAgo',
 'PR2SalesTTM',
 'EarnYield',
 'Pr2BookQ',
 'Pr2TanBkQ',
 'FCFYield',
 '$unlevered_fcfToEV',
 'OcfToEV',
 'EV2SalesTTM',
 'GrossProfitToEV',
 'EV2EBITDATTM',
 'EPSReportedPctGrPYQ',
 'EPSReportedAccelPYQTTM',
 'EPSReportedPctGrTTM',
 'SalesActualGr%PYQ',
 'SalesActualGr%TTM',
 'SalesActualAccelPYQTTM',
 'DbtTot2CapQ',
 'IntCovTTM',
 'AstTurnTTM',
 'OpMgn%TTM',
 'ROI%Q',
 'ROE%Q',
 'Div%ChgTTM',
 'Yield',
 'net_payout_ratio',
 'CurFYEPS_pctchg4WK',
 'CurQEPS_pctchg4WK',
 'CurQEPS_estvar',
 'CurFYEPS_estvar',
 'SalesSurp%Q1',
 'Surprise%Q1',
 'fwd_ebitdantm_ev',
 'fwd_ebitdaq_ev',
 'epsg_pyq_est_act',
 'DATE']

In [11]:
shift_cols = [ c for c in df.columns if c not in ['ID', 'DATE', 'stock_return'] ]

In [12]:
dfnan_counts = df.groupby(['DATE']).apply(lambda x: x.isna().sum())

/var/folders/sw/kl2bvz9d5275zz0p278kz9nc0000gn/T/ipykernel_97285/3733439746.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dfnan_counts = df.groupby(['DATE']).apply(lambda x: x.isna().sum())


In [13]:
df = df.sort_values(['DATE','ID'])

fdf = df.set_index(['DATE','ID'])

In [14]:
# make sure ID and DATE are set as index before you do this 
fdf[shift_cols] = fdf.groupby('ID')[shift_cols].shift(1)

In [15]:
shifted_dates = fdf.index.get_level_values('DATE').unique()[:1]

In [16]:
fdf = fdf[~fdf.index.get_level_values('DATE').isin(shifted_dates)]

### functions - rolling reg / var

### calculate simple returns 

In [17]:
fdf = fdf.reset_index()

In [18]:
first_dates = df.apply(lambda x: x.first_valid_index())

first_dates = first_dates.rename('first_idx').to_frame()
value_columns = df.columns.difference(['ID', 'DATE'])

# For each column
dseries = pd.DataFrame()
for column in value_columns:
    first_date = df[df[column].notna()]['DATE'].min()
    last_date = df[df[column].notna()]['DATE'].max()
    first_dates.loc[column,'first_date'] = first_date
    first_dates.loc[column,'last_date'] = last_date

In [19]:
fdf = fdf.set_index(['ID', 'DATE'])

In [20]:
idx = pd.IndexSlice
cutoff_date = '2015-12-31'
fdf = fdf[fdf.index.get_level_values('DATE') > cutoff_date]

In [21]:
fdf[value_columns] = fdf.groupby('DATE')[value_columns].transform(lambda x: x.fillna(x.mean()))

In [22]:
fdfnan_counts = fdf.groupby(['DATE']).apply(lambda x: x.isna().sum())

In [23]:
Ycols = value_columns.difference(['stock_return'])

In [24]:
def calculate_bounded_trimmed_zscore(data, trim_percent=1, bounds=(-5, 5)):
    """
    Calculate z-scores using trimmed statistics and bound them
    
    Parameters:
    data: pandas.Series, Input data
    trim_percent: float, Percentage to trim from each end (default 1%)
    bounds: tuple, (lower_bound, upper_bound) for z-scores
    
    Returns:
    pandas.Series: Bounded z-scores
    """
    # Calculate trimmed statistics
    lower_percentile = np.nanpercentile(data, trim_percent)
    upper_percentile = np.nanpercentile(data, 100 - trim_percent)
    
    trimmed_data = data[(data >= lower_percentile) & (data <= upper_percentile)]
    
    trim_mean = np.nanmean(trimmed_data)
    trim_std = np.nanstd(trimmed_data)
    
    # Check for zero variance or NaN
    if trim_std == 0 or np.isnan(trim_std):
        return pd.Series(np.full_like(data, np.nan), index=data.index)
    
    # Calculate z-scores
    z_scores = (data - trim_mean) / trim_std
    
    # Bound z-scores
    bounded_zscores = np.clip(z_scores, bounds[0], bounds[1])
    
    return bounded_zscores #pd.Series(bounded_zscores, index=data.index)



In [27]:
columns_to_apply_zscore = [
'MktCap',
'Beta1Y',
'price_vol_1y',
'share_turnover_3m',
'residual_return_1m',

'3MoRS',
'3MoRS3MoAgo',
'3MoRS6MoAgo',
'3MoRS9MoAgo',
]

In [28]:
trim_percent=1
bounds=(-5, 5)

fdf[columns_to_apply_zscore] = fdf.groupby('DATE')[columns_to_apply_zscore].transform(
    lambda x: calculate_bounded_trimmed_zscore(x, trim_percent, bounds)
)

In [29]:
from scipy.stats.mstats import winsorize
from scipy.stats import zscore

# from p123 its already zscored, so no need to do anything, returns are not zscores 

# df[numcols] = df[numcols].groupby('ITERATION_DATE').transform(lambda x: winsorize(x, limits = (0.01,0.01)))
# fdf[Ycols] = fdf[Ycols].groupby('DATE').transform(lambda x: zscore(x).clip(-3,3))


In [30]:
def cross_sectional_regression(df, dependent_var, independent_vars, reg_type='OLS', alpha=1.0):
    """
    Performs regression in parallel for each group using joblib, with options for OLS, Ridge, and Lasso.
    Returns np.nan for coefficients when X or y contains NaN values.

    Args:
        df: Pandas DataFrame containing the data with columns ID, DATE, and other variables
        dependent_var: Name of the dependent variable column
        independent_vars: List of names of independent variable columns
        reg_type: Type of regression to perform ('OLS', 'Ridge', 'Lasso'). Default is 'OLS'
        alpha: Regularization strength for Ridge and Lasso. Default is 1.0

    Returns:
        Pandas DataFrame with the regression coefficients for each ID and DATE
    """
    from sklearn.linear_model import LinearRegression, Ridge, Lasso
    from joblib import Parallel, delayed
    import numpy as np
    import pandas as pd
    
    def _regress_group(group_data):
        X = group_data[independent_vars].values
        y = group_data[dependent_var].values.reshape(-1,1)
        
        # Return NaN coefficients if data contains NaN
        if np.any(np.isnan(X)) or np.any(np.isnan(y)):
            return pd.Series({
                'DATE': group_data['DATE'].iloc[0],
                'intercept': np.nan,
                **dict(zip(independent_vars, [np.nan] * len(independent_vars)))
            })
            
        if reg_type.upper() == 'OLS':
            model = LinearRegression()
        elif reg_type.upper() == 'RIDGE':
            model = Ridge(alpha=alpha)
        elif reg_type.upper() == 'LASSO':
            model = Lasso(alpha=alpha)
        else:
            raise ValueError("Invalid reg_type. Choose 'OLS', 'Ridge', or 'Lasso'.")
            
        model.fit(X, y)
        coefs = model.coef_.flatten()
        intercept = model.intercept_
        
        return pd.Series({
            'DATE': group_data['DATE'].iloc[0],
            'intercept': intercept[0] if isinstance(intercept, np.ndarray) else intercept,
            **dict(zip(independent_vars, coefs))
        })
    
    # Group by DATE and perform regression
    results = Parallel(n_jobs=-1)(
        delayed(_regress_group)(group) 
        for _, group in df.groupby('DATE')
    )
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    results_df = results_df.set_index('DATE')
    
    return results_df

# Let's create an example to demonstrate the usage

In [31]:
fdf = fdf.reset_index()

In [33]:
# fdf.query('DATE == "2024-01-06"')
# fdf[fdf['DATE'] > '2024-06-01']['DATE'].unique()
# fdf.query('DATE == "2024-06-22"')

In [34]:
betas = cross_sectional_regression(fdf, dependent_var='stock_return', independent_vars=Ycols)

In [35]:
betas = betas.rolling(12).mean()


In [36]:

betas = betas.reset_index().set_index('DATE')
betas.index = pd.to_datetime(betas.index)

In [37]:
betas_shifted = pd.concat([
    betas.shift(1),
    pd.DataFrame([betas.iloc[-1]], index=[betas.index[-1] + pd.offsets.MonthEnd(1)])
])
betas_shifted.index.name = 'DATE'

In [38]:
# for manual confirmation 
betas_shifted[list(Ycols) + ['intercept']].to_csv('betas_p123.csv')
# fdf[['ID','DATE'] + list(Ycols)].to_csv('fdf_p123.csv')

In [40]:
fdf.to_csv('fdf_p123.csv')

In [41]:
fdf = fdf.set_index(['ID','DATE'])

In [57]:
# fdf[Ycols].assign(intercept = 1).mul(betas_shifted, level = 'DATE').query('DATE == "2024-04-06"').sum(axis = 1)
# yfit.query('DATE == "2024-04-06"').sum(axis = 1)

In [87]:
yfit = fdf[Ycols].assign(intercept = 1).mul(betas_shifted, level = 'DATE').sum(axis = 1).rename('exp_ret').to_frame()

In [88]:
yfit = yfit[yfit.index.get_level_values('DATE')> pd.Timestamp('2024-03-30')]

In [89]:
yfit['quantile'] = yfit.groupby('DATE', group_keys = False)['exp_ret'].apply(lambda x: pd.qcut(x, 10, labels = False)).rename('quantile')

In [76]:
# using stats as of date 
# Dec 1st stats (shifted) with Dec end returns gives Dec beta. 
# avearge of Jan to Dec betas gives Dec average beta 
# Dec beta with Dec returns gives dec ranks 

In [90]:
yfit = yfit.merge(fdf[['stock_return']], left_index = True, right_index = True)

In [91]:
# this is ignoring a lot of the features, but need this for now to proceed with just model fitting 
# goal is daily data, 


In [92]:
yfit.to_csv('../p123/data/nootc_2024_features.csv')

In [93]:
# what output do we need?
# 1. betas 
# 2. tstats
# 3. ranks and realized returns for decile testing => goes to rank backtester 
# 4. current portfolio 
# 5. can redo ranks based on combining top betas differently 


In [94]:
yfit

,,exp_ret,quantile,stock_return
ID,DATE,,,
A,2024-04-06,0.007463,6,-0.007944
AA,2024-04-06,-0.012204,0,0.079905
AACG,2024-04-06,0.007186,6,0.000094
AACT,2024-04-06,0.014230,9,0.000473
AAGR,2024-04-06,0.010018,8,-0.226718
...,...,...,...,...
ZVRA,2025-03-29,-0.003954,5,-0.019975
ZVSA,2025-03-29,-0.011595,1,-0.183624
ZWS,2025-03-29,-0.003083,5,-0.014943


In [95]:
yfit.groupby(['DATE']).apply(lambda x: len(x['quantile'].unique()))#.head(20)

DATE
2024-04-06    10
2024-04-13    10
2024-04-20    10
2024-04-27    10
2024-05-04    10
2024-05-11    10
2024-05-18    10
2024-05-25    10
2024-06-01    10
2024-06-08    10
2024-06-15    10
2024-06-22    10
2024-06-29    10
2024-07-06    10
2024-07-13    10
2024-07-20    10
2024-07-27    10
2024-08-03    10
2024-08-10    10
2024-08-17    10
2024-08-24    10
2024-08-31    10
2024-09-07    10
2024-09-14    10
2024-09-21    10
2024-09-28    10
2024-10-05    10
2024-10-12    10
2024-10-19    10
2024-10-26    10
2024-11-02    10
2024-11-09    10
2024-11-16    10
2024-11-23    10
2024-11-30    10
2024-12-07    10
2024-12-14    10
2024-12-21    10
2024-12-28    10
2025-01-04    10
2025-01-11    10
2025-01-18    10
2025-01-25    10
2025-02-01    10
2025-02-08    10
2025-02-15    10
2025-02-22    10
2025-03-01    10
2025-03-08    10
2025-03-15    10
2025-03-22    10
2025-03-29    10
dtype: int64

In [ ]:
yfit.groupby(['DATE','quantile'])['exp_ret'].count().unstack()

quantile,0,1,2,3,4,5,6,7,8,9
DATE,,,,,,,,,,
2024-04-06,556,555,555,555,555,555,555,555,555,556
2024-04-13,555,554,555,554,555,554,554,555,554,555
2024-04-20,555,555,555,555,555,554,555,555,555,555
2024-04-27,555,555,555,554,555,555,554,555,555,555
2024-05-04,555,554,555,554,555,554,554,555,554,555
2024-05-11,554,554,554,554,554,553,554,554,554,554
2024-05-18,554,553,553,554,553,553,554,553,553,554
2024-05-25,553,552,553,552,553,552,552,553,552,553
2024-06-01,553,552,552,552,553,552,552,552,552,553


In [98]:
yfit.groupby(['DATE','quantile'])['stock_return'].mean().unstack()

quantile,0,1,2,3,4,5,6,7,8,9
DATE,,,,,,,,,,
2024-04-06,-0.028777,-0.024738,-0.013502,-0.011295,-0.020302,-0.003886,-0.015755,-0.011508,-0.006393,-0.011695
2024-04-13,-0.023473,-0.028658,-0.030415,-0.028307,-0.025823,-0.026955,-0.021490,-0.025293,-0.024293,-0.026164
2024-04-20,-0.047609,-0.023771,-0.023785,-0.024221,-0.020769,-0.025387,-0.027701,-0.037197,-0.041657,-0.056065
2024-04-27,0.026595,0.020763,0.018229,0.027846,0.024610,0.018470,0.008432,0.022203,0.013034,0.020209
2024-05-04,0.032953,0.025140,0.024520,0.024063,0.014884,0.022233,0.017484,0.015573,0.009752,0.013942
2024-05-11,-0.016712,-0.005186,0.003606,0.009827,0.002347,0.016211,0.011547,0.009490,0.011678,0.016880
2024-05-18,0.080316,0.031853,0.056519,0.022380,0.018355,0.005664,0.015889,0.019376,0.015705,0.016289
2024-05-25,-0.032943,-0.017666,-0.014349,-0.011297,-0.021252,-0.014006,-0.011529,-0.012208,-0.015227,-0.012914
2024-06-01,0.013739,-0.003758,-0.003941,-0.001762,-0.000083,-0.005596,0.002899,0.006921,0.004440,0.006000


In [102]:
yfit.query('DATE == DATE.max()').sort_values(by = 'exp_ret', ascending = False).head(40)

,,exp_ret,quantile,stock_return
ID,DATE,,,
YRD,2025-03-29,0.047721,9,0.050432
UCL,2025-03-29,0.038411,9,-0.023437
GLBS,2025-03-29,0.022016,9,-0.032284
CCLD,2025-03-29,0.019789,9,-0.026490
ABTS,2025-03-29,0.018495,9,-0.066296
MOMO,2025-03-29,0.017262,9,-0.038052
CMLS,2025-03-29,0.016960,9,0.048526
ASBP,2025-03-29,0.016654,9,0.015228
OXBR,2025-03-29,0.014860,9,-0.069124
